In [ ]:
from IPython.core.debugger import Tracer

import keras
import keras.losses
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, load_model, Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import keras.backend as K

from scipy import io
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import random
import csv
import os

import socket
import socketserver
import time
import threading
from IPython.display import clear_output

np.set_printoptions(precision=10, suppress=True)
def my_mape(y_true, y_pred):
    return K.sum((K.abs(y_pred - y_true))/(K.abs(y_true)+0.001))
keras.losses.my_mape = my_mape

# serverSocket =socket.socket(socket.AF_INET,socket.SOCK_STREAM)
# serverSocket.bind (('127.0.0.1',50000))
# serverSocket.listen(1)

Q_max = np.genfromtxt('./MATLAB/data/Generate/temp_0430_0.csv', delimiter=',')[:,20:35].max(axis = 0)
Q_min = np.genfromtxt('./MATLAB/data/Generate/temp_0430_0.csv', delimiter=',')[:,20:35].min(axis = 0)
def Q_normalize(Q_num):
    return (Q_num - Q_min)/(Q_max - Q_min)

def Q_denormalize(X_pred_nor):
    res = X_pred_nor.copy()
    Freq = res[:,2:20]
    res[:,2:20] = np.power(10, Freq)
    Q_num = res[:,20:]
    Q_num = Q_num*(Q_max - Q_min)+Q_min
    res[:,20:] = Q_num
    return res

In [ ]:
X_in_ = []
X_out_ = []
Y_in_ = []
Y_out_ = []
indices_ = []

In [ ]:
# Load datas
directory_name = './MATLAB/data/0304/'
# data_amount = 7500

X_DesignedFs = []
X_poles = []
X_Qnums = []
X_LQ = []
# X_rks = []
# X_yks = []
# X_uks = []
X_eks = []
Z_DesignedFs = []
Z_poles = []
Z_Qnums = []
Z_LQ = []
# Z_rks = []
# Z_yks = []
# Z_uks = []
Z_eks = []
for plant_dir in ['X/', 'Z/']:
    files_name = os.listdir(directory_name+plant_dir)
    print(directory_name+plant_dir, len(files_name), 'files')
    for file in files_name[::2]:
#     for file in files_name[:data_amount]:
        load_data = io.loadmat(directory_name + plant_dir + file)
        plant = file[0]
        DesignedF = load_data[plant+'_DesignedF']
        pole = load_data[plant+'_pole'].reshape(-1)[0]
        Qnum = load_data[plant+'_Qnum'].reshape(-1)
        LQ = Qnum.shape[0]
#         rk = (load_data[plant+'_rk'].reshape(-1))[7500:22500]
#         yk = (load_data[plant+'_yk'].reshape(-1))[7500:22500]
#         uk = (load_data[plant+'_uk'].reshape(-1))[7500:22500]
        ek = (load_data[plant+'_ek'].reshape(-1))[7500:22500]
    #     print(file, ek.max())
        if ek.max() < 25:
            if plant == 'X':
                X_DesignedFs.append(DesignedF)
                X_poles.append(pole)
                X_Qnums.append(Qnum)
                X_LQ.append(LQ)
#                 X_rks.append(rk)
#                 X_yks.append(yk)
#                 X_uks.append(uk)
                X_eks.append(ek)
            elif plant == 'Z':
                Z_DesignedFs.append(DesignedF)
                Z_poles.append(pole)
                Z_Qnums.append(Qnum)
                Z_LQ.append(LQ)
#                 Z_rks.append(rk)
#                 Z_yks.append(yk)
#                 Z_uks.append(uk)
                Z_eks.append(ek)
X_poles = np.array(X_poles)
X_LQ = np.array(X_LQ)
# X_rks = np.array(X_rks)
# X_yks = np.array(X_yks)
# X_uks = np.array(X_uks)
X_eks = np.array(X_eks)
X_Qnums = np.array(X_Qnums)
X_DesignedFs = np.array(X_DesignedFs)[:,:,1]
Z_poles = np.array(Z_poles)
Z_LQ = np.array(Z_LQ)
# Z_rks = np.array(Z_rks)
# Z_yks = np.array(Z_yks)
# Z_uks = np.array(Z_uks)
Z_eks = np.array(Z_eks)
Z_Qnums = np.array(Z_Qnums)
Z_DesignedFs = np.array(Z_DesignedFs)[:,:,1]

Q_max = X_Qnums.max(axis = 0)
Q_min = X_Qnums.min(axis = 0)
print('finish')
X_in_.append(np.hstack((X_poles.reshape(-1, 1), X_LQ.reshape(-1, 1), np.log10(X_DesignedFs))))
X_out_.append(Q_normalize(X_Qnums))
Y_in_.append(np.append(X_in_[-1], X_out_[-1], axis = 1))
Y_out_.append(X_eks.max(axis = 1))
indices_.append(np.arange(X_in_[-1].shape[0]))
np.random.shuffle(indices_[-1])
print(X_in_[-1].shape, X_out_[-1].shape, Y_in_[-1].shape, Y_out_[-1].shape)

In [ ]:
X_in_ = []
X_out_ = []
Y_in_ = []
Y_out_ = []
indices_ = []
for idx in [0, 1, 3, 4, 5, 6]:
    p_LQ = np.genfromtxt('./MATLAB/data/Generate/temp_0430_{}.csv'.format(idx), delimiter=',')[:,:2]
    Freq = np.log10(np.genfromtxt('./MATLAB/data/Generate/temp_0430_{}.csv'.format(idx), delimiter=',')[:,2:20])
    Q = Q_normalize(np.genfromtxt('./MATLAB/data/Generate/temp_0430_{}.csv'.format(idx), delimiter=',')[:,20:35])
    Y_in_.append(np.hstack((p_LQ, Freq, Q)))
    Y_out_.append(np.genfromtxt('./MATLAB/data/Generate/temp_error_0430_{}.csv'.format(idx), delimiter=',')[:,0]*1000)
    print(Y_in_[-1].shape, Y_out_[-1].shape)
    indices_.append(np.arange(Y_in_[-1].shape[0]))
    np.random.shuffle(indices_[-1])


In [ ]:
X_Qnums[::50].shape

In [ ]:
with open('.\workspace\data\X_Q_0607.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(X_Qnums[::50])
with open('.\workspace\data\Z_Q_0607.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(Z_Qnums[::50])

In [ ]:
p_LQ = np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,:2]
Freq = np.log10(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,2:20])
Q = Q_normalize(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,20:35])
Y_in_.append(np.hstack((p_LQ, Freq, Q)))
Y_out_.append(np.genfromtxt('./MATLAB/data/Generate/temp_error.csv', delimiter=',')[:,0]*1000)
print(Y_in_[-1].shape, Y_out_[-1].shape)
indices_.append(np.arange(Y_in_[-1].shape[0]))
np.random.shuffle(indices_[-1])

with open('./MATLAB/data/Generate/temp_0430_5.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=','))
with open('./MATLAB/data/Generate/temp_error_0430_5.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(np.genfromtxt('./MATLAB/data/Generate/temp_error.csv', delimiter=','))

In [ ]:
with open('./MATLAB/data/Generate/temp_0430_0.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(Q_denormalize(Y_in_[-1]))

In [ ]:
Y = np.append(Y_in_[0], Y_in_[1],axis = 0)
Z = np.append(Y_out_[0], Y_out_[1],axis = 0)
Z = np.append(Y_out_[0], Y_out_[1],axis = 0)
print(Y.shape, Z.shape)

In [ ]:
Y = []
Z = []
for idx in range(6):
    p_LQ = np.genfromtxt('./MATLAB/data/Generate/temp_0430_{}.csv'.format(idx), delimiter=',')[:,:2]
    Freq = np.log10(np.genfromtxt('./MATLAB/data/Generate/temp_0430_{}.csv'.format(idx), delimiter=',')[:,2:20])
    Q = Q_normalize(np.genfromtxt('./MATLAB/data/Generate/temp_0430_{}.csv'.format(idx), delimiter=',')[:,20:35])
    Y.append(np.hstack((p_LQ, Freq, Q)))
    Z.append(np.genfromtxt('./MATLAB/data/Generate/temp_error_0430_{}.csv'.format(idx), delimiter=',')[:,0]*1000)
    print(Y[-1].shape, Z[-1].shape)
#     indices_.append(np.arange(Y_in_[-1].shape[0]))
#     np.random.shuffle(indices_[-1])


In [ ]:
np.genfromtxt('./MATLAB/data/Generate/temp_error_0430_{}.csv'.format(0), delimiter=',')

In [ ]:
(Y_in_[0][20:35] - Y_in_[-1][20:35])

In [ ]:
X_in = Y_in_[0][:,:20]
X_out = Y_in_[0][:,20:35]
Y_in = Y_in_[0]
Y_out = Y_out_[0]
for i in range(1,len(Y_out_)):
    Y_in = np.append(Y_in, Y_in_[i], axis = 0)
    Y_out = np.append(Y_out, Y_out_[i], axis = 0)
print(X_in.shape, X_out.shape, Y_in.shape, Y_out.shape)
indices = np.arange(Y_in.shape[0])
np.random.shuffle(indices)

fig = plt.figure(figsize=(15, 16))
ax1 = host_subplot(211, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(Y_in)[:,i+20], alpha = 0.5)
    ax1.set_title('Original Q num')

ax1 = host_subplot(212, axes_class=AA.Axes)
ax1.plot(Y_out, alpha = 0.5)
ax1.set_title('Original Err')
# ax1.set_ylim(0,10)
plt.show()


In [ ]:
X_in = Y[0][:20]
X_out = Y[0][20:35]
Y_in = Y[0]
Y_out = Z[0]
for i in [1, 2, 3, 4, 5]:
    Y_in = np.append(Y_in, Y[i], axis = 0)
    Y_out = np.append(Y_out, Z[i], axis = 0)
print(X_in.shape, X_out.shape, Y_in.shape, Y_out.shape)
indices = np.arange(Y_in.shape[0])
np.random.shuffle(indices)

fig = plt.figure(figsize=(15, 16))
ax1 = host_subplot(211, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(Y_in)[:,i+20], alpha = 0.5)
    ax1.set_title('Original Q num')

ax1 = host_subplot(212, axes_class=AA.Axes)
ax1.plot(Y_out, alpha = 0.5)
ax1.set_title('Original Err')
# ax1.set_ylim(0,10)
plt.show()

# fig = plt.figure(figsize=(15, 8))
# ax1 = host_subplot(111, axes_class=AA.Axes)
# ax1.plot(Y_out_[-1], alpha = 0.5)
# ax1.set_title('Original Err')
# # ax1.set_ylim(0,10)
# plt.show()

In [ ]:
X_pred = self_G.predict(X_in_[0])
with open('./MATLAB/data/Generate/temp.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(Q_denormalize(X_pred))
print("start matlab")
os.system(r"matlab -r \"run('C:\Users\dalek669528\Desktop\CNC_proj\MATLAB\Simulation_FixedQ_Ccode\X_main_fixedQ_Ccode_func.m')\"")
print("open socket")
connectionSocket, addr = serverSocket.accept()

In [ ]:
len(Y)

In [ ]:

print(X_in_[0].shape, X_out_[0].shape, Y_in_[0].shape, Y_out_[0].shape)

In [ ]:
keras.backend.clear_session()

In [ ]:
def __generator():
    """ Declare generator """
    G_in = Input(shape = (X_in.shape[1],))
    L1 = Dense(units=64, kernel_initializer='normal')(G_in)
    L1 = BatchNormalization()(L1)
    L1 = Dense(units = 128, activation='sigmoid')(L1)
#     L1 = Dropout(0.1)(L1)
#     L1 = BatchNormalization(momentum=0.8)(L1)
    L1 = Dense(units = 64)(L1)
    G_out = Dense(units = X_out.shape[1])(L1)
    G_out = Concatenate(axis = -1)([G_in, G_out])
    model = Model(inputs = G_in, outputs = G_out)
    model.name = "G"
    return model

In [ ]:
def __discriminator():
    """ Declare discriminator """
    model = Sequential()
    model.add(Dense(units=64, input_dim=Y_in.shape[1], kernel_initializer='normal'))
    model.add(Dropout(0.1))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(units=128, kernel_initializer='normal', activation='sigmoid'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(units=64, activation='sigmoid'))
#     model.add(Dropout(0.1))
    model.add(Dense(units=1))
    model.name = "D"
    return model

In [ ]:
def __stacked_generator_discriminator(self_G, self_D):
    self_D.trainable = False
    model = Sequential()
    model.add(self_G)
    model.add(self_D)
    model.name = "GD"
    return model

In [ ]:
self_D = __discriminator()
self_D.compile(loss='mae', optimizer=Adam(lr=0.0001, beta_1=0.5, decay=8e-8))
self_D.summary()

In [ ]:
self_G = __generator()
self_G.compile(loss=my_mape, optimizer=Adam(lr=0.0001, beta_1=0.5, decay=8e-8))
self_G.summary()

In [ ]:
self_stacked_GD = __stacked_generator_discriminator(self_G, self_D)
self_stacked_GD.compile(loss='mae', optimizer=Adam(lr=0.0002, beta_1=0.5, decay=8e-8))
self_stacked_GD.summary()

In [ ]:
self_D.trainable = True
self_D.compile(loss='mae', optimizer=Adam(lr=0.00002, beta_1=0.5, decay=8e-8))
train_history_D = self_D.fit(x=Y_in[indices], y=Y_out[indices], validation_split=0.5, 
                              epochs=100000, batch_size=256, verbose=1, 
                              callbacks = [EarlyStopping(monitor = 'val_loss', patience = 1000, verbose = 1, mode = 'auto')])


In [ ]:
self_D.trainable = False
self_stacked_GD.compile(loss='mae', optimizer=Adam(lr=0.000001, beta_1=0.5, decay=8e-8))
train_history_GD = self_stacked_GD.fit(x=X_in_[0][indices_[0]], y=Y_out_[0][indices_[0]], validation_split=0.3,
                                        epochs=100000, batch_size=256, verbose=1, 
                                        callbacks = [EarlyStopping(monitor = 'val_loss', patience = 2000, verbose = 1, mode = 'auto')])

In [ ]:
self_stacked_GD.test_on_batch(X_in_[0], Y_out_[0])

In [ ]:
self_stacked_GD.train_on_batch(X_in_[0], Y_out_[0])

In [ ]:
train_history_D = []
train_history_GD = []

for idx in range(50):

    X_pred = self_G.predict(X_in)
    
    with open('./MATLAB/data/Generate/temp.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(Q_denormalize(X_pred))
    print("start matlab")
#     os.system(r"matlab -r \"run('C:\Users\dalek669528\Desktop\CNC_proj\MATLAB\Simulation_FixedQ_Ccode\X_main_fixedQ_Ccode_func.m')\"")
#     print("open socket")
#     connectionSocket, addr = serverSocket.accept()
    with open('./MATLAB/data/Generate/check_{}.csv'.format(idx), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(Q_denormalize(X_pred))
    self_G.save('./models/checkpoint/G_{}.h5'.format(idx))
    self_D.save('./models/checkpoint/D_{}.h5'.format(idx))
    self_stacked_GD.save('./models/checkpoint/StackGD_{}.h5'.format(idx))
    indices_2 = np.arange(Y_in_2.shape[0])
    np.random.shuffle(indices_2)
    self_D.trainable = True
    self_D.compile(loss='mae', optimizer=Adam(lr=0.00001, beta_1=0.5, decay=8e-8))
    train_history_D.append(self_D.fit(x=Y_in_2[indices_2], y=Y_out_2[indices_2], validation_split=0.3, 
                                      epochs=10000, batch_size=256, verbose=2, 
                                      callbacks = [EarlyStopping(monitor = 'val_loss', patience = 1000+idx*500, verbose = 1, mode = 'auto')]))
    self_D.trainable = False
    self_stacked_GD.compile(loss='mape', optimizer=Adam(lr=0.000001, beta_1=0.5, decay=8e-8))
    train_history_GD.append(self_stacked_GD.fit(x=X_in[indices], y=Y_out[indices], shuffle=True, validation_split=0.2, 
                                                epochs=100000, batch_size=256, verbose=2, 
                                                callbacks = [EarlyStopping(monitor = 'val_loss', patience = 1000+idx*500, verbose = 1, mode = 'auto')]))

    
#     connectionSocket, addr = serverSocket.accept()
#     sentence = connectionSocket.recv(1024)
#     print(sentence.decode())
#     connectionSocket.close() 
    
#     p_LQ = np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,:2]
#     Freq = np.log10(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,2:20])
#     Q = Q_normalize(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,20:35])
#     Y_in_resimulate = np.hstack((p_LQ, Freq, Q))
#     Y_out_resimulate = np.genfromtxt('./MATLAB/data/Generate/temp_error.csv', delimiter=',')[:,1]*1000
#     Y_in_2 = np.append(Y_in_2, Y_in_resimulate, axis=0)
#     Y_out_2 = np.append(Y_out_2, Y_out_resimulate)
    print(Y_in_2.shape, Y_out_2.shape)
    clear_output(wait=True)

In [ ]:
# # z_mislabled = np.zeros((X_train.shape[0], 1))
# self_D.trainable = False
# self_stacked_GD.compile(loss='mape', optimizer=Adam(lr=0.000001, beta_1=0.5, decay=8e-8))
# train_history_GD = self_stacked_GD.fit(x=X_in[indices], y=Y_out[indices], shuffle=True, validation_split=0.7, epochs=100000, batch_size=256, verbose=1, callbacks = [callback])


In [ ]:
# self_G = load_model('./models/Simulator_DesignF2Qnum_0430_7.h5') 
# self_G.name = "G"
# self_G.compile(loss=my_mape, optimizer=Adam(lr=0.0000001, beta_1=0.5, decay=8e-8))
# self_G.summary()
self_D = load_model('./models/Simulator_Qnum2Errmax_0430_7.h5')
self_D.name = "D"
self_D.compile(loss='mape', optimizer=Adam(lr=0.000001, beta_1=0.5, decay=8e-8))
self_D.summary()

# self_stacked_GD = __stacked_generator_discriminator(self_G, self_D)
# self_stacked_GD.compile(loss='mae', optimizer=Adam(lr=0.0002, beta_1=0.5, decay=8e-8))
# self_stacked_GD.summary()

In [ ]:
self_G.save('./models/Simulator_DesignF2Qnum_0430_9.h5')
self_D.save('./models/Simulator_Qnum2Errmax_0430_9.h5')
self_stacked_GD.save('./models/Stacked_GAN_0430_9.h5')

In [ ]:
X_pred = self_G.predict(X_in)
Y_pred = self_stacked_GD.predict(X_in)
Y_pred_D = self_D.predict(Y_in_[-2])
fig = plt.figure(figsize=(15, 16))
ax1 = host_subplot(211, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(Y_in_[-2])[:,i+20], alpha = 0.5)
    ax1.set_title('Original Q num')
ax1 = host_subplot(212, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(X_pred)[:,i+20], alpha = 0.5)
    ax1.set_title('Original Q num')
plt.show()

fig = plt.figure(figsize=(15, 20))
ax1 = host_subplot(311, axes_class=AA.Axes)
ax1.plot(Y_out_[-2], alpha = 0.5)
ax1.set_title('Original Err')
ax1 = host_subplot(312, axes_class=AA.Axes)
ax1.plot(Y_pred, alpha = 0.5)
ax1.set_title('Original Err')
ax1 = host_subplot(313, axes_class=AA.Axes)
ax1.plot(Y_pred_D, alpha = 0.5)
ax1.set_title('Original Err')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 16))
ax1 = host_subplot(211, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(Y_in_[-1])[:,i+20], alpha = 0.5)
    ax1.set_title('Original Q num')

ax1 = host_subplot(212, axes_class=AA.Axes)
ax1.plot(Y_out_[-1], alpha = 0.5)
ax1.set_title('Original Err')
plt.show()


In [ ]:
np.all(X_in[indices_[0]] == X_in_[0])

In [ ]:
X_pred = self_G.predict(X_in_[0])
Y_pred = self_stacked_GD.predict(X_in_[0])
Y_pred_D = self_D.predict(Y_in)
fig = plt.figure(figsize=(15, 16))
ax1 = host_subplot(211, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(Y_in_[0])[:,i+20], alpha = 0.5)
    ax1.set_title('Original Q num')
# plt.show()
# fig = plt.figure(figsize=(15, 8))
ax1 = host_subplot(212, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(X_pred)[:,i+20], alpha = 0.8)
    ax1.set_title('Predict Q num')
plt.show()

fig = plt.figure(figsize=(15, 8))
ax1 = host_subplot(111, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(Y_in_[-1])[:,i+20], alpha = 0.8)
    ax1.set_title('Predict Q num')
plt.show()


fig = plt.figure(figsize=(15, 8))
ax1 = host_subplot(111, axes_class=AA.Axes)
ax1.plot(Y_out_[0][indices_[0]], alpha = 1, label = "Original error")
ax1.plot(Y_pred, alpha = 0.5, label = "Predict error")
ax1.set_title('max error')
ax1.legend()
# ax1.set_ylim(0, 1)
plt.show()
fig = plt.figure(figsize=(15, 8))
ax1 = host_subplot(111, axes_class=AA.Axes)
ax1.plot(Y_out_[0][500:1000], alpha = 1)
ax1.plot(Y_pred[500:1000], alpha = 0.5)
# ax1.set_ylim(0, 1)
ax1.set_title('max error')
plt.show()

fig = plt.figure(figsize=(15, 8))
ax1 = host_subplot(111, axes_class=AA.Axes)
ax1.plot(Y_out, alpha = 1, label = "Original error")
ax1.plot(Y_pred_D, alpha = 0.3, label = "Predict error")
ax1.set_title('max error')
ax1.legend()
# ax1.set_ylim(0, 1)
plt.show()
# fig = plt.figure(figsize=(15, 8))
# ax1 = host_subplot(111, axes_class=AA.Axes)
# ax1.plot(Y_out[500:1000], alpha = 1)
# ax1.plot(Y_pred_D[500:1000], alpha = 0.3)
# # ax1.set_ylim(0, 1)
# ax1.set_title('max error')
# plt.show()

In [ ]:
# X_pred = self_G.predict(X_in)
fig = plt.figure(figsize=(15, 8))
ax1 = host_subplot(111, axes_class=AA.Axes)
    
for i in range(15):
    ax1.plot(Q_denormalize(Y_in_[-1])[:,i+20], alpha = 0.8)
    ax1.set_title('Real Q num')
#     ax1.plot(Q_denormalize(X_pred)[:,i+20], alpha = 0.8)
#     ax1.set_title('Predict Q num')
plt.show()
    
# fig = plt.figure(figsize=(15, 8))
# ax1 = host_subplot(111, axes_class=AA.Axes)
# for i in range(15):
#     ax1.plot(X_pred[:,i], alpha = 1)
#     ax1.set_title('Predict Q num')
# plt.show()

In [ ]:
# Y_test = self_D.predict(Y_in)
fig = plt.figure(figsize=(15, 16))
ax1 = host_subplot(211, axes_class=AA.Axes)
ax1.plot(Y_out_, alpha = 0.9)
# ax1.plot(Y_test, alpha = 0.5)
# ax1 = host_subplot(212, axes_class=AA.Axes)
# ax1.plot(Y_out_3[:200], alpha = 0.9)
# ax1.plot(Y_test[:200], alpha = 0.5)
plt.show()

In [ ]:
np.array(Y_out_[])